In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
color = sns.color_palette()

import datetime
import math
import gc

%matplotlib inline

In [2]:
drop_duplicate = True
drop_sim_feat = False
use_corr = True
use_prep_day = True
use_cross = True

In [3]:
path = "./data/"
train_data = pd.read_table(path + 'train.txt',sep=" ")
test_data = pd.read_table(path + 'test.txt',sep=" ")
test_data['is_trade'] = 0
train_data = pd.concat([train_data,test_data],ignore_index=True)

In [4]:
if drop_duplicate:
    train_data = train_data.drop_duplicates()

In [5]:
train_data["datetime"] = train_data["context_timestamp"].apply(lambda x: datetime.datetime.fromtimestamp(x))

In [6]:
train_data["day"] = train_data["datetime"].apply(lambda x: x.day)
train_data["hour"] = train_data["datetime"].apply(lambda x: x.hour)

In [ ]:
缺失值填充

In [9]:
for column in ['shop_review_positive_rate','shop_score_service', 'shop_score_delivery','shop_score_description']:
    train_data.loc[train_data[column] == -1,column] = train_data.loc[train_data[column] != -1,column].mean()

In [10]:
train_data.loc[train_data['user_gender_id'] == -1,'user_gender_id'] = 0

In [10]:
train_data.loc[train_data['user_age_level'] == -1,'user_age_level'] = 1003

In [11]:
train_data.loc[train_data['user_occupation_id'] == -1,'user_occupation_id'] = 2005

In [12]:
train_data.loc[train_data['user_star_level'] == -1,'user_star_level'] = 3006

按时间排序

In [7]:
train_data = train_data.sort_values('context_timestamp')

In [8]:
train_data["item_category_list"] = train_data["item_category_list"].apply(lambda x: x.split(";"))
train_data["item_property_list"] = train_data["item_property_list"].apply(lambda x: x.split(";"))

In [9]:
categories = train_data["predict_category_property"].apply(lambda x: x.split(";"))
train_data["num_query_cat"] = categories.apply(lambda x: len(x))
for i in range(categories.apply(lambda x: len(x)).max()):
    train_data["category_"+str(i)] = categories.apply(lambda x: x[i].split(":")[0] if len(x)>i else "-1")
    train_data["category_"+str(i)+"_props"] = categories.apply(lambda x: x[i].split(":")[1].split(",") if len(x)>i and x[i].split(":")[0] != "-1" else ["-1"])

In [10]:
user_query_day = train_data.groupby(['user_id', 'day']).size().reset_index().rename(columns={0: 'user_query_day'})
train_data = pd.merge(train_data, user_query_day, 'left', on=['user_id', 'day'])
user_query_day_hour = train_data.groupby(['user_id', 'day', 'hour']).size().reset_index(
                      ).rename(columns={0: 'user_query_day_hour'})
train_data = pd.merge(train_data, user_query_day_hour, 'left',on=['user_id', 'day', 'hour'])

修改后用cat_prop_corr cat_corr

In [11]:
def get_query_corr(df):
    data = df[[ 'category_0',
       'category_0_props', 'category_1', 'category_1_props', 'category_2',
       'category_2_props', 'category_3', 'category_3_props', 'category_4',
       'category_4_props', 'category_5', 'category_5_props', 'category_6',
       'category_6_props', 'category_7', 'category_7_props', 'category_8',
       'category_8_props', 'category_9', 'category_9_props', 'category_10',
       'category_10_props', 'category_11', 'category_11_props', 'category_12',
       'category_12_props', 'category_13', 'category_13_props','item_property_list','item_category_list',
        'num_query_cat','cat_prop_corr','cat_corr']].values
    
    for list in data:
        count1 = 0
        count2 = 0
        for num in range(list[-3]):
            if list[num*2] != "-1" and list[num*2] in list[-4]:
                count1 = count1 + 1 + list[-4].index(list[num*2])
                for prop in list[num*2+1]:
                    if prop != "-1" and prop in list[-5]:
                        count2 = count2 + 1
        list[-1] = count1
        list[-2] = count2

In [12]:
if use_corr:
    train_data["cat_corr"] = 0
    train_data["cat_prop_corr"] = 0
    get_query_corr(train_data)

In [13]:
train_data["num_item_category"] = train_data["item_category_list"].apply(lambda x: len(x)-x.count("-1"))
train_data["num_item_property"] = train_data["item_property_list"].apply(lambda x: len(x)-x.count("-1"))

In [14]:
if drop_sim_feat:
    train_data = train_data.drop(["shop_score_delivery", "shop_star_level", "shop_score_description", "user_star_level" , "item_collected_level"], axis=1)  

生成id信息的历史日志

In [15]:
def create_history(df,column):
    df[column + '_click_history']=''

    dict={}
    data = df[['instance_id',column,'is_trade',column+'_click_history']].values

    for list in data:
        user = list[1]
        if user in dict:
            if dict[user][0] == list[0]:
                list[3] = dict[user][3]
            else:
                list[3] =dict[user][3] + str(dict[user][2])
                dict[user] = list
        else:
                dict[user] = list
    dict.clear()
    
    df[['instance_id',column,'is_trade',column+'_click_history']] = data

In [16]:
for column in ['user_id','item_id','item_brand_id','shop_id']:
    create_history(train_data,column)

In [17]:
for column in ['user_id','item_id','item_brand_id','shop_id']:
    train_data[column+'_click_count_prep'] = train_data[column+'_click_history'].apply(lambda x: len(x))

统计商品被点击的主要影响属性

In [19]:
trade_data = train_data.loc[train_data['is_trade']==1,:].copy()

In [20]:
def get_query_item_props_and_cats(data):
    for i in range(data.shape[0]):
        props = ''
        cats =''
        for num in range(data.loc[i,"num_query_cat"]):
            if data.loc[i,"category_"+str(num)] != "-1" and data.loc[i,"category_"+str(num)] in data.loc[i,"item_category_list"]:
                if cats == '':
                    cats = data.loc[i,"category_"+str(num)]
                else:
                    cats = cats+'_'+data.loc[i,"category_"+str(num)]
                for prop in data.loc[i,"category_"+str(num)+"_props"]:
                    if prop != "-1" and prop in data.loc[i,"item_property_list"]:
                        if props == '':
                            props = prop
                        else:
                            props = props+'_'+prop
        data.loc[i,"item_props"] = props
        data.loc[i,"item_cats"] = cats

In [21]:
trade_data['item_props'] = ''

In [22]:
trade_data = trade_data.reset_index()

In [23]:
get_query_item_props_and_cats(trade_data)

In [24]:
trade_data["item_props"] = trade_data["item_props"].apply(lambda x: x.split('_'))

In [25]:
def listToDict(x):
    dict = {}
    for i in range(len(x)):
        if x[i] != '':
            if x[i] in dict:
                dict[x[i]] += 1
            else:
                dict[x[i]] = 1
    return dict

In [26]:
train_data = train_data.join(trade_data.groupby('item_id')['item_props'].sum(),on = 'item_id')

In [27]:
train_data['item_props'] = train_data['item_props'].apply(lambda x: listToDict(x) if x is not np.nan else {})

在此次样本出现前用户可曾看过此广告，可曾trade过

In [28]:
train_data['user_item_id'] = train_data['user_id'].astype(str)+"_"+train_data['item_id'].astype(str)

In [29]:
train_data['has_seen'] = 0
train_data['has_trade'] = 0

In [30]:
train_data['has_trade'] = train_data['user_item_id']+':'+train_data['is_trade'].astype(str)
train_data['has_trade'] = train_data['has_trade'].apply(lambda x: x.split(":"))

In [31]:
dict_has_seen = {}
def has_seen(x,dict):
    if x not in dict:
        dict[x] = 0
    else:
        dict[x] += 1
    return dict[x]

In [32]:
yy = train_data['user_item_id'].values

In [33]:
i = 0
for list in yy:
    yy[i] = has_seen(list,dict_has_seen)
    i += 1

In [34]:
train_data['has_seen'] = train_data['user_item_id'].values

In [35]:
train_data['has_seen'] = train_data['has_seen'].astype('int64')

In [36]:
dict2 = {}
def has_trade(x,dict):
    if x[0] not in dict:
        if x[1]=='1':
            dict[x[0]] = 1
        else:
            dict[x[0]] = 0
        return 0
    else:
        if x[1]=='1':
            dict[x[0]] += 1
            return dict[x[0]] - 1
        else:
            return dict[x[0]]

In [37]:
yy = train_data['has_trade'].values

In [38]:
i = 0
for list in yy:
    yy[i] = has_trade(list,dict2)
    i += 1

In [39]:
train_data['item_category_0'] = train_data['item_category_list'].apply(lambda x: x[0])
train_data['item_category_1'] = train_data['item_category_list'].apply(lambda x: x[1])
train_data['item_category_2'] = train_data['item_category_list'].apply(lambda x: x[2] if len(x)>2 else 0)

统计总count数

In [40]:
for column in ['user_id','item_id','item_brand_id','shop_id']:
    train_data = train_data.join(train_data[column].value_counts(),on = column ,rsuffix = '_count')

对id类进行划分

In [41]:
def cat_user(x):
    if x>=5 and x<=7:
        return '5-7'
    if x>7:
        return '>7'
    else:
        return str(x)

In [42]:
train_data['user_id_click_count_prep_cat'] = train_data['user_id_click_count_prep'].apply(lambda x: cat_user(x))

In [43]:
def cut(x,list):
    for i in range(len(list)):
        if x<=list[i]:
            return i
    return len(list)

In [74]:
for column in ['item_id','item_brand_id','shop_id']:
    list_quantile = []
    for i in range(1,21,1):
        list_quantile.append(train_data[column+'_click_count_prep'].quantile(0.05*i))
    train_data[column+'_click_count_prep_cat'] = train_data[column+'_click_count_prep'].apply(lambda x: cut(x,list_quantile))

In [75]:
def cut_id(x,list,thresh):
    if x<thresh:
        for i in range(len(list)):
            if x<=list[i]:
                return i
    else:
        return 'self'

In [ ]:
for column in ['item_id','item_brand_id','shop_id']:
    list_quantile = []
    for i in range(1,21,1):
        list_quantile.append(train_data[column+'_count'].quantile(0.05*i))
    train_data[column+'_count_cat'] = train_data[column+'_count'].apply(lambda x: cut_id(x,list_quantile,500))
    data = train_data[[column,column+'_count_cat']].values
    for list in data:
        if list[1] == 'self':
            list[1] = list[0]
    train_data[[column,column+'_count_cat']] = data

In [77]:
list_quantile = []
for i in range(1,21,1):
    list_quantile.append(train_data['user_id_count'].quantile(0.05*i))
train_data['user_id_count_cat'] = train_data['user_id_count'].apply(lambda x: cut_id(x,list_quantile,10))
data = train_data[['user_id','user_id_count_cat']].values
for list in data:
    if list[1] == 'self':
        list[1] = list[0]
train_data[['user_id','user_id_count_cat']] = data

统计样本之前的trade数

In [41]:
def trade_count(x):
    sum = ''
    for i in range(len(x)):
        if x[i] == '1':
            sum += x[i]
    return sum

In [42]:
for column in ['user_id','item_id','item_brand_id','shop_id']:
    train_data[column+'_trade_count'] = train_data[column+'_click_history'].apply(lambda x: len(trade_count(x)))

商品最关键属性

In [43]:
train_data['key_prop'] = 0
data = train_data[['category_0','category_0_props', 'category_1', 'category_1_props', 'category_2',
       'category_2_props', 'category_3', 'category_3_props', 'category_4',
       'category_4_props', 'category_5', 'category_5_props', 'category_6',
       'category_6_props', 'category_7', 'category_7_props', 'category_8',
       'category_8_props', 'category_9', 'category_9_props', 'category_10',
       'category_10_props', 'category_11', 'category_11_props', 'category_12',
       'category_12_props', 'category_13', 'category_13_props', 'key_prop', 'item_props','item_category_list','num_query_cat']].values

In [44]:
for list in data:
    for i in range(list[-1]):
        if list[2*i] in list[-2]:
            for prop in list[2*i+1]:
                if prop in list[-3]:
                    list[-4] = max(list[-3][prop],list[-4])

可改进 此处可使用prop或比值

In [45]:
train_data[['category_0','category_0_props', 'category_1', 'category_1_props', 'category_2',
       'category_2_props', 'category_3', 'category_3_props', 'category_4',
       'category_4_props', 'category_5', 'category_5_props', 'category_6',
       'category_6_props', 'category_7', 'category_7_props', 'category_8',
       'category_8_props', 'category_9', 'category_9_props', 'category_10',
       'category_10_props', 'category_11', 'category_11_props', 'category_12',
       'category_12_props', 'category_13', 'category_13_props', 'key_prop', 'item_props','item_category_list','num_query_cat']] = data

In [46]:
def prep_count(df,group_column,column):
    dict_group = {}
    data = df[[group_column,column]].values
    for list in data:
        if list[0] not in dict_group:
            dict_group[list[0]] = list[1]
        else:
            list[1] = dict_group[list[0]]
    df[[group_column,column]] = data

In [47]:
if use_prep_day:
    for i in range(18,26,1):
        data = train_data.loc[train_data['day'] == i,:].copy()
        prep_count(data,'user_item_id','has_trade')
        train_data.loc[train_data['day'] == i,:] = data
    
    for i in range(18,26,1):
        data = train_data.loc[train_data['day'] == i,:].copy()
        prep_count(data,'user_id','user_id_trade_count')
        train_data.loc[train_data['day'] == i,:] = data
    
    for i in range(18,26,1):
        data = train_data.loc[train_data['day'] == i,:].copy()
        prep_count(data,'item_id','item_id_trade_count')
        train_data.loc[train_data['day'] == i,:] = data
    
    for i in range(18,26,1):
        data = train_data.loc[train_data['day'] == i,:].copy()
        prep_count(data,'item_brand_id','item_brand_id_trade_count')
        train_data.loc[train_data['day'] == i,:] = data
    
    for i in range(18,26,1):
        data = train_data.loc[train_data['day'] == i,:].copy()
        prep_count(data,'shop_id','shop_id_trade_count')
        train_data.loc[train_data['day'] == i,:] = data

利用query分析用户偏好，并与商品类别属性进行交叉

In [48]:
def get_cats(x):
    list = x.split(';')
    catlist = []
    for entry in list:
        catlist.append(entry.split(':')[0])
    return catlist

In [49]:
def create_cat_props(x):
    list = x.split(';')
    dict={}
    if x == '-1':
        return dict
    for entry in list:
        cat_props = entry.split(':')
        dict[cat_props[0]] = cat_props[1].split(',')
    return dict

In [50]:
def get_weight(x):
    sum = 0
    for key in x:
        sum = max(sum,x[key])
    return sum

In [51]:
if use_cross:
    train_data['user_int_cats'] = train_data['predict_category_property'].apply(lambda x: get_cats(x))
    
    data = train_data[['user_id','user_int_cats']].values
    dict_0 = {}
    for list in data:
        if list[0] not in dict_0:
            to_dict = {}
        else:
            to_dict = dict_0[list[0]]
        for cat in list[1]:
            if cat not in to_dict:
                to_dict[cat] = 1
            else:
                to_dict[cat] += 1
        dict_0[list[0]] = to_dict
        
    train_data['user_int_cat'] = train_data['user_id'].apply(lambda x: dict_0[x])
    
    train_data['category_property'] = train_data['predict_category_property'].apply(lambda x: create_cat_props(x))

    data = train_data[['user_id','category_property']].values
    dict_0.clear()
    for list in data:
        if list[0] not in dict_0:
            to_dict = {}
        else:
            to_dict = dict_0[list[0]]
        for cat in list[1]:
            if cat not in to_dict:
                to_dict[cat] = {}
                for prop in list[1][cat]:
                    if prop != '-1':
                        to_dict[cat][prop] = 1
            else:
                for prop in list[1][cat]:
                    if prop != '-1':
                        if prop in to_dict[cat]:
                            to_dict[cat][prop] += 1
                        else:
                            to_dict[cat][prop] = 1
        dict_0[list[0]] = to_dict
    
    train_data['user_int_props'] = train_data['user_id'].apply(lambda x: dict_0[x])
    dict_0.clear()
    train_data['user_item_cross_prop'] = ''
    train_data['user_item_cross_cat'] = ''
    
    data = train_data[['item_category_list','user_int_cat','user_item_cross_cat',
                   'item_property_list','user_int_props','user_item_cross_prop']].values
    for list in data:
        list[2] = {}
        list[-1] = {}
        for cat in list[0]:
            if cat != '7908382889764677758':
                if cat in list[1]:
                    list[2][cat] = list[1][cat]
            for prop in list[3]:
                if cat in list[4] and prop in list[4][cat]:
                    list[-1][prop] = list[4][cat][prop]
                    
    train_data[['item_category_list','user_int_cat','user_item_cross_cat',
                   'item_property_list','user_int_props','user_item_cross_prop']] = data
    
    train_data['cross_prop_num'] = train_data.user_item_cross_prop.apply(lambda x: len(x))
    train_data['cross_cat_num'] = train_data.user_item_cross_cat.apply(lambda x: len(x))
    train_data['cross_prop_weight'] = train_data.user_item_cross_prop.apply(lambda x: get_weight(x))
    train_data['cross_cat_weight'] = train_data.user_item_cross_cat.apply(lambda x: get_weight(x))

In [29]:
train_data[['cross_prop_num','cross_cat_num','cross_prop_weight','cross_cat_weight','key_prop']].to_csv(path+'cross_query.csv')

计算类别ctr（非id类）

In [52]:
def count_ctr(df,day,column):
    ctr_dict = {}
    for i in df.loc[df.day<day,column].unique():
        ctr_dict[i] = df.loc[np.logical_and(df[column] == i,df.day<day),'is_trade'].mean()
    return ctr_dict

In [53]:
def ctr(df,day,ctr_count_list):
    for column in ctr_count_list:
        ctr_dict = count_ctr(df,day,column)
        df[column+'_ctr'] = df[column].apply(lambda x: ctr_dict[x] if x in ctr_dict else -1)

In [ ]:
ctr(train_data,25,['item_price_level', 'item_sales_level',
                 'item_pv_level', 'user_gender_id', 'user_occupation_id',
                'user_age_level', 
                'context_page_id', 'hour', 'shop_review_num_level',
            "shop_star_level",  "user_star_level" , "item_collected_level",
        'item_category_1', 'item_category_2','cross_prop_num'])

In [ ]:
gc.collect()

将数据转化成libsvm格式

In [58]:
def pd_to_libsvm(df):    
    libsvm = pd.DataFrame()  
    idx = 0  
    t = df.dtypes.to_dict()    
    for col in df.columns:  
        col_type = t[col]  
        if col_type ==  'object':
            col_value = df[col].unique()  
            feat_dict = dict(zip(col_value,range(idx,idx+len(col_value))))  
            libsvm[col] = df[col].apply(lambda x: str(feat_dict[x]) + ':' + '1')  
            idx += len(col_value) 
        else:
            libsvm[col] = df[col].apply(lambda x: str(idx) + ':' + str(x))
            idx += 1
    return libsvm

In [59]:
def to_libsvm_file(data,path,test_day,valid_day = -1):    
    y = data['is_trade']
    day_split = data['day']
    x = data.drop(['is_trade','day'],axis=1)
    ffm_x = pd_to_libsvm(x)
    ffm = pd.DataFrame()
    ffm['is_trade'] = y
    ffm = ffm.merge(ffm_x,left_index=True,right_index=True)
    if valid_day != -1:    
        train = ffm.loc[np.logical_and(day_split != valid_day,day_split != test_day),:]
        valid = ffm.loc[day_split == valid_day,:]
        valid.to_csv(path+'_valid.txt', sep = ' ', header = False, index=False)
    else:
        train = ffm.loc[day_split != test_day,:]
    test = ffm.loc[day_split == test_day,:]
    test.to_csv(path+'_test.txt', sep = ' ', header = False, index=False)
    train.to_csv(path+'_train.txt', sep = ' ', header = False, index=False)

选择训练新user的特征

In [60]:
features = ['item_id','item_brand_id','item_city_id','shop_id','item_price_level', 'item_sales_level',
                 'item_pv_level', 'user_gender_id', 'user_occupation_id',
                'user_age_level', 
                'context_page_id', 'hour',  'shop_review_num_level',
                'shop_review_positive_rate', 'shop_score_service',"shop_score_delivery",
            "shop_star_level", "shop_score_description", "user_star_level" , "item_collected_level",
           'num_query_cat', 'cat_prop_corr',
        'num_item_property','user_id_click_count_prep',
       'item_id_click_count_prep', 'item_brand_id_click_count_prep',
       'shop_id_click_count_prep', 'has_seen',
       'has_trade','item_category_1', 'item_category_2', 'item_id_count', 'item_brand_id_count',
       'shop_id_count','user_id_trade_count', 'item_id_trade_count',
       'item_brand_id_trade_count', 'shop_id_trade_count','cross_prop_num', 'cross_prop_weight','user_id_count']

In [ ]:
选择训练老用户user的特征

In [72]:
features = ['item_id','item_brand_id','item_city_id','shop_id','item_price_level', 'item_sales_level',
                 'item_pv_level', 'user_gender_id', 'user_occupation_id',
                'user_age_level', 'user_query_day', 'user_query_day_hour',
                'context_page_id', 'hour',  'shop_review_num_level',
                'shop_review_positive_rate', 'shop_score_service',"shop_score_delivery",
            "shop_star_level", "shop_score_description", "user_star_level" , "item_collected_level",
           'num_query_cat', 'cat_prop_corr',
        'num_item_property',
       'item_id_click_count_prep', 'item_brand_id_click_count_prep',
       'shop_id_click_count_prep','item_category_1', 'item_id_count', 'item_brand_id_count',
       'shop_id_count', 'item_id_trade_count',
       'item_brand_id_trade_count', 'shop_id_trade_count']

In [62]:
ctr_column = [i+'_ctr' for i in ['item_price_level', 'item_sales_level',
                 'item_pv_level', 'user_gender_id', 'user_occupation_id',
                'user_age_level', 
                'context_page_id', 'hour', 'shop_review_num_level',
            "shop_star_level",  "user_star_level" , "item_collected_level",
        'item_category_1','cross_prop_num']]

In [ ]:
train_data[(features+['is_trade','day']+ctr_column)].info()

In [ ]:
gc.collect()

In [65]:
to_libsvm_file(train_data[features+['is_trade','day']+ctr_column],path+'baseline_0',25,-1)

In [ ]:
to_libsvm_file(train_data.loc[train_data.user_id_click_count_prep!=0,features+['is_trade','day']+ctr_column],path+'baseline_1_old',25,-1)
to_libsvm_file(train_data.loc[train_data.user_id_click_count_prep==0,features+['is_trade','day']+ctr_column],path+'baseline_1_new',25,-1)